### Predicting Disease based on the Symptoms and identifying specialist based on predicted disease

**The dataset contains information about symptoms, diseases, recommended doctors for each disease, and descriptions of the diseases. Using a classification model, diseases have been predicted based on the given symptoms. The predictions provide insights into the predicted disease, the likelihood or chance of that disease based on the classification model, the recommended doctor to visit for that disease, and a description of the predicted disease.**

**The summary output includes:**

**Predicted Disease:** *-The disease that has been predicted based on the symptoms provided.-*

**Chance of the Disease:** *-The likelihood or probability of having that disease, as determined by the classification model.-*

**Recommended Doctor:** *-The doctor or specialist recommended to visit for further evaluation or treatment of the predicted disease.-*

**Description of the Disease:** *-A brief overview or description of the predicted disease, providing additional information about its symptoms, causes, and potential treatments.-*

**This summary aims to provide a concise and informative overview of the predicted disease, its associated likelihood, the recommended doctor, and a brief description to aid in understanding and decision-making regarding the predicted disease.**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from collections import Counter

In [2]:
pd.set_option('display.max_colwidth', -1)

In [3]:
dis_sym_data = pd.read_csv("/kaggle/input/doctor-specialist-recommendation-system/Original_Dataset.csv")

In [4]:
dis_sym_data.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fungal infection,itching,skin_rash,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
dis_sym_data.shape

(4920, 18)

**Finding unique values across all the symptoms column**

In [6]:
columns_to_check = []
for col in dis_sym_data.columns:
    if col != 'Disease':
        columns_to_check.append(col)

In [7]:
symptoms = dis_sym_data.iloc[:, 1:].values.flatten()
symptoms = list(set(symptoms))

**Convert Symptoms to Binary Columns**

In [8]:
for symptom in symptoms:
    dis_sym_data[symptom] = dis_sym_data.iloc[:, 1:].apply(lambda row: int(symptom in row.values), axis=1)

dis_sym_data_v1 = dis_sym_data.drop(columns=columns_to_check)

In [9]:
dis_sym_data_v1 = dis_sym_data_v1.loc[:, dis_sym_data_v1.columns.notna()]

In [10]:
dis_sym_data_v1.shape

(4920, 132)

In [11]:
dis_sym_data_v1.columns = dis_sym_data_v1.columns.str.strip()

In [12]:
dis_sym_data_v1.columns

Index(['Disease', 'receiving_blood_transfusion', 'dark_urine', 'coma',
       'movement_stiffness', 'passage_of_gases', 'swelling_joints',
       'neck_pain', 'skin_peeling', 'yellow_urine',
       ...
       'history_of_alcohol_consumption', 'cramps', 'muscle_wasting',
       'sinus_pressure', 'patches_in_throat', 'foul_smell_of urine',
       'continuous_sneezing', 'spotting_ urination', 'mucoid_sputum',
       'pain_behind_the_eyes'],
      dtype='object', length=132)

**Encoding Disease**

In [13]:
var_mod = ['Disease']
le = LabelEncoder()
for i in var_mod:
    dis_sym_data_v1[i] = le.fit_transform(dis_sym_data_v1[i])

In [14]:
X = dis_sym_data_v1.drop(columns="Disease")
y = dis_sym_data_v1['Disease']

**Model Generation**

In [15]:
def class_algo(model,independent,dependent):
    model.fit(independent,dependent)
    pred = model.predict(independent)
    accuracy = metrics.accuracy_score(pred,dependent)
    print(model_name,'Accuracy : %s' % '{0:.3%}'.format(accuracy))

In [16]:
algorithms = {'Logistic Regression': 
              {"model": LogisticRegression()},
              
              'Decision Tree': 
              {"model": tree.DecisionTreeClassifier()},
              
              'Random Forest': 
              {"model": RandomForestClassifier()},
              
              'SVM':
              {"model": svm.SVC(probability=True)},
              
              'NaiveBayes' :
              {"model": GaussianNB()},
              
              'K-Nearest Neighbors' :
              {"model": KNeighborsClassifier()},
             }

In [17]:
for model_name, values in algorithms.items():
    class_algo(values["model"],X,y)

Logistic Regression Accuracy : 100.000%

Decision Tree Accuracy : 100.000%

Random Forest Accuracy : 100.000%

SVM Accuracy : 100.000%

NaiveBayes Accuracy : 100.000%

K-Nearest Neighbors Accuracy : 100.000%


**Map Description and Specialist for the Disease Predicted**

In [18]:
doc_data = pd.read_csv("/kaggle/input/doctor-specialist-recommendation-system/Doctor_Versus_Disease.csv",encoding='latin1', names=['Disease','Specialist'])

In [19]:
doc_data.tail(5)

,Disease,Specialist
36,Bronchial Asthma,Pulmonologist
37,Pneumonia,Pulmonologist
38,Osteoarthristis,Rheumatologists
39,Arthritis,Rheumatologists
40,Tuberculosis,Tuberculosis


 -*Doctor name is wrong for Disease - Tuberculosis. Replace to Pulmonologist*-

In [20]:
doc_data['Specialist'] = np.where((doc_data['Disease'] == 'Tuberculosis'),'Pulmonologist', doc_data['Specialist'])

In [21]:
doc_data.tail(5)

,Disease,Specialist
36,Bronchial Asthma,Pulmonologist
37,Pneumonia,Pulmonologist
38,Osteoarthristis,Rheumatologists
39,Arthritis,Rheumatologists
40,Tuberculosis,Pulmonologist


In [22]:
des_data = pd.read_csv("/kaggle/input/doctor-specialist-recommendation-system/Disease_Description.csv")

In [23]:
des_data.head()

,Disease,Description
0,Drug Reaction,An adverse drug reaction (ADR) is an injury caused by taking medication. ADRs may occur following a single dose or prolonged administration of a drug or result from the combination of two or more drugs.
1,Malaria,An infectious disease caused by protozoan parasites from the Plasmodium family that can be transmitted by the bite of the Anopheles mosquito or by a contaminated needle or transfusion. Falciparum malaria is the most deadly type.
2,Allergy,"An allergy is an immune system response to a foreign substance that's not typically harmful to your body.They can include certain foods, pollen, or pet dander. Your immune system's job is to keep you healthy by fighting harmful pathogens."
3,Hypothyroidism,"Hypothyroidism, also called underactive thyroid or low thyroid, is a disorder of the endocrine system in which the thyroid gland does not produce enough thyroid hormone."
4,Psoriasis,"Psoriasis is a common skin disorder that forms thick, red, bumpy patches covered with silvery scales. They can pop up anywhere, but most appear on the scalp, elbows, knees, and lower back. Psoriasis can't be passed from person to person. It does sometimes happen in members of the same family."


**Test with Unknown Data**

In [24]:
test_col = []
for col in dis_sym_data_v1.columns:
    if col != 'Disease':
        test_col.append(col)


test_data = {}
symptoms = []
predicted = []
def test_input():
    symptoms.clear()
    predicted.clear()
    num_inputs = int(input("Enter the number of symptoms you have: "))
    for i in range(num_inputs):
        user_input = input("Enter Symptoms #{}: ".format(i+1))
        symptoms.append(user_input)
    print("Symptoms you have:", symptoms)
    for column in test_col:
        test_data[column] = 1 if column in symptoms else 0
        test_df = pd.DataFrame(test_data, index=[0])
    print("Predicting Disease based on 6 ML algorithms...")
    for model_name, values in algorithms.items():
        predict_disease = values["model"].predict(test_df)
        predict_disease = le.inverse_transform(predict_disease)
        predicted.extend(predict_disease)
    disease_counts = Counter(predicted)
    percentage_per_disease = {disease: (count / 6) * 100 for disease, count in disease_counts.items()}
    result_df = pd.DataFrame({"Disease": list(percentage_per_disease.keys()),
                               "Chances": list(percentage_per_disease.values())})
    result_df = result_df.merge(doc_data, on='Disease', how='left')
    result_df = result_df.merge(des_data, on='Disease', how='left')
    return result_df

In [25]:
test_input()

Enter the number of symptoms you have: 3

Enter Symptoms #1: chills

Enter Symptoms #2: knee_pain

Enter Symptoms #3: acidity

Symptoms you have: ['chills', 'knee_pain', 'acidity']

Predicting Disease based on 6 ML algorithms...


,Disease,Chances,Specialist,Description
0,Allergy,100.0,Allergist,"An allergy is an immune system response to a foreign substance that's not typically harmful to your body.They can include certain foods, pollen, or pet dander. Your immune system's job is to keep you healthy by fighting harmful pathogens."
